In [1]:
#%pip install yfinance

In [23]:
import yfinance as yf
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

from scipy.stats import norm

In [ ]:
# Chargement de l'historique des actifs sur les deux dernières années
end_date = datetime.now()
strat_date = end_date - timedelta(days=2*365)
tickers = ['SAN.PA', 'BN.PA',  'AIR.PA', 'AI.PA', 'ORA.PA', 'CAP.PA', 'VIV.PA', 'CA.PA', 'ENGI.PA', 'DG.PA']
df = yf.download(tickers, start = strat_date, end=end_date)[['Close', 'Volume']]

cac40 = yf.download('^FCHI', start = strat_date, end=end_date)[['Close', 'Volume']]

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
df.head()

Price            Close                                                \
Ticker           AI.PA      AIR.PA      BN.PA      CA.PA      CAP.PA   
Date                                                                   
2023-01-30  129.703018  113.202339  46.732342  16.069134  166.543777   
2023-01-31  128.141388  112.535980  46.928230  16.005003  167.555466   
2023-02-01  127.790459  113.711906  47.292015  15.972937  170.349670   
2023-02-02  128.702881  115.397400  47.124111  16.078293  181.478287   
2023-02-03  127.492165  114.770248  47.329327  15.995840  182.104553   

Price                                                             Volume  \
Ticker          DG.PA    ENGI.PA    ORA.PA     SAN.PA    VIV.PA    AI.PA   
Date                                                                       
2023-01-30  96.130051  10.756802  8.415674  83.109291  9.450280   922207   
2023-01-31  95.871040  10.796604  8.508443  82.998215  9.362902   982086   
2023-02-01  96.870087  10.906063  8.504067  82.100380  9.395194   498535   
2023-02-02  97.166107  10.707047  8.443680  80.313980  9.469277   935729   
2023-02-03  97.443604  10.859626  8.393794  78.805267  9.402792  1217643   

Price                                                                     \
Ticker       AIR.PA    BN.PA    CA.PA  CAP.PA    DG.PA  ENGI.PA   ORA.PA   
Date                                                                       
2023-01-30  1080203  1318109  1503551  265113   832568  3784922  5398200   
2023-01-31  1357678  1195913  1947261  472465   850875  5761592  7767232   
2023-02-01  1102143  1111493  1091097  272322   907425  5894306  4980785   
2023-02-02  1394176  2013542  1805955  568976   920499  7742563  7100632   
2023-02-03  1587465  1649190  1730754  383622  1106022  5942379  7368347   

Price                         
Ticker       SAN.PA   VIV.PA  
Date                          
2023-01-30  1235322  2938565  
2023-01-31  2218706  2470382  
2023-02-01  1614583  2025489  
2023-02-02  2421074  2434350  
2023-02-03  4394526  2855250

In [5]:
df_close = df['Close']
df_volume = df['Volume']

In [6]:
# Poids des actions dans le portefeuille
weights = np.repeat(0.1, 10)

weights = pd.DataFrame({'Ticker': ['SAN.PA', 'BN.PA',  'AIR.PA', 'AI.PA', 'ORA.PA', 'CAP.PA', 'VIV.PA', 'CA.PA', 'ENGI.PA', 'DG.PA'],
                        'Weight': weights})
weights = weights.set_index('Ticker')['Weight']

print(weights)

Ticker
SAN.PA     0.1
BN.PA      0.1
AIR.PA     0.1
AI.PA      0.1
ORA.PA     0.1
CAP.PA     0.1
VIV.PA     0.1
CA.PA      0.1
ENGI.PA    0.1
DG.PA      0.1
Name: Weight, dtype: float64


In [7]:
# Détermination de l'AUM
aum = df_close@weights
portfolio = pd.DataFrame({'AUM': aum})
portfolio.head()

,AUM
Date,
2023-01-30,68.011271
2023-01-31,67.870327
2023-02-01,68.289278
2023-02-02,69.488106
2023-02-03,69.259722


In [8]:
# Performance du portefeuille
portfolio['perf'] = portfolio['AUM'].pct_change()
portfolio.dropna(inplace=True)


In [9]:
portfolio.head()

,AUM,perf
Date,,
2023-01-31,67.870327,-0.002072
2023-02-01,68.289278,0.006173
2023-02-02,69.488106,0.017555
2023-02-03,69.259722,-0.003287
2023-02-06,68.508216,-0.010851


# Volatiliy

In [10]:
# Volatilité annualisé du portefeuille
std_dev = portfolio.std()*np.sqrt(252)
std_dev

AUM     74.059264
perf     0.127592
dtype: float64

In [11]:
# Performance du benchmark
cac40_df = cac40['Close']
cac40_df ['perf'] = cac40_df.pct_change()
cac40_df.dropna(inplace=True)


In [12]:
# Volatilité du benchmark
cac40_df.std()*np.sqrt(252)

Ticker
^FCHI    4951.982443
perf        0.133750
dtype: float64

# Tracking error

In [13]:
# Tracking error
portfolio['benchmark'] = cac40_df['perf']
portfolio['tracking_error'] = portfolio['perf'] - portfolio['benchmark']
TError = portfolio['tracking_error'].std()*np.sqrt(252) # Annualisation

TError

0.07699809526025904

# Value at risk

## VaR historique

In [38]:
VaR = portfolio.quantile(1-0.99)*np.sqrt(20) # 1-day VaR skaled to 20-day VaR
print(VaR)

AUM               305.467473
perf               -0.102666
benchmark          -0.095200
tracking_error     -0.056828
Name: 0.010000000000000009, dtype: float64


## VaR paramétrique

In [36]:
VaR = (portfolio.mean() + norm.ppf(1-0.99)*portfolio.std())*np.sqrt(20)
print(VaR)

AUM               296.048988
perf               -0.081962
benchmark          -0.086554
tracking_error     -0.049907
dtype: float64


# Stress test

In [64]:
# Données des actions en période covid
stress_date_1 = "2020-02-19"
stress_date_2 = "2020-03-19"

tickers = ['SAN.PA', 'BN.PA',  'AIR.PA', 'AI.PA', 'ORA.PA', 'CAP.PA', 'VIV.PA', 'CA.PA', 'ENGI.PA', 'DG.PA']
df_stress = yf.download(tickers, start = stress_date_1, end=stress_date_2)['Close']
cac40_stress = yf.download('^FCHI', start = stress_date_1, end=stress_date_2)['Close']

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


In [65]:
aum_stress = df_stress@weights
portfolio_stress = pd.DataFrame({'AUM': aum_stress})


In [66]:
# Rendement du portefeuille en période de covid
portfolio_stress.iloc[-1,] /portfolio_stress.iloc[0,]  - 1

AUM   -0.394026
dtype: float64

In [67]:
# Rendemnt du benchmark enn période covid
cac40_stress.iloc[-1,] / cac40_stress.iloc[0,] -1

Ticker
^FCHI   -0.385585
dtype: float64